# PACS-AI Assist — Volume Viewer (LIDC-IDRI)

Este notebook carga un **volumen CT 3D** desde tu subset de LIDC-IDRI y ofrece:
- Sliders **axial, coronal, sagital** (con `ipywidgets`).
- **Cine loop** (animación) axial.
- Filtro robusto para elegir una **serie con PixelData**.

> Ajusta `BASE_DIR` solo si moviste la carpeta `data/`. El resto se autodetecta.


In [2]:
# Ejecuta una sola vez por entorno (.venv)
%pip install -q ipywidgets==8.1.2 jupyterlab-widgets==3.0.10 numpy matplotlib pydicom


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pathlib import Path
import pydicom, numpy as np, matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, HBox
from collections import defaultdict

# 📁 ajusta solo si cambiaste la ruta base
BASE_DIR = Path("data/LIDC-IDRI_subset")

# --- Autodetección de manifest/LIDC-IDRI ---
manifests = sorted([p for p in BASE_DIR.iterdir() if p.is_dir() and p.name.startswith("manifest-")])
assert manifests, f"No encontré 'manifest-*' dentro de {BASE_DIR.resolve()}"
DATA_ROOT = manifests[-1] / "LIDC-IDRI"

patients = [p for p in DATA_ROOT.iterdir() if p.is_dir() and p.name.startswith("LIDC-IDRI-")]
assert patients, f"No hay carpetas de pacientes dentro de {DATA_ROOT}"
print("Pacientes disponibles:", ", ".join(p.name for p in patients))


In [ ]:
def pick_series_with_pixels(patient_dir: Path):
    series = defaultdict(list)
    for fp in patient_dir.rglob("*.dcm"):
        try:
            ds = pydicom.dcmread(fp, stop_before_pixels=True, force=True)
            uid = getattr(ds, "SeriesInstanceUID", None)
            if uid:
                series[uid].append(fp)
        except Exception:
            pass

    def good(files):
        for f in files[:2]:
            try:
                ds = pydicom.dcmread(f, force=True)
                _ = ds.pixel_array
                return True
            except Exception:
                pass
        return False

    for uid, files in sorted(series.items(), key=lambda kv: -len(kv[1])):
        if good(files):
            return uid, files
    raise RuntimeError("No encontré series con PixelData en este paciente.")

def order_key(fp: Path):
    ds = pydicom.dcmread(fp, stop_before_pixels=True, force=True)
    ipp = getattr(ds, "ImagePositionPatient", None)
    if ipp and len(ipp) == 3:
        return float(ipp[2])
    inst = getattr(ds, "InstanceNumber", None)
    return (inst is None, inst, str(fp))

def load_volume_from_patient(patient_dir: Path):
    uid, files = pick_series_with_pixels(patient_dir)
    files = sorted(files, key=order_key)
    ex = pydicom.dcmread(files[0], force=True)
    slope = float(getattr(ex, "RescaleSlope", 1.0) or 1.0)
    intercept = float(getattr(ex, "RescaleIntercept", 0.0) or 0.0)
    vol = np.stack([pydicom.dcmread(f, force=True).pixel_array.astype(np.float32)*slope+intercept for f in files], axis=0)
    spacing = getattr(ex, "PixelSpacing", [1,1])
    slice_thk = float(getattr(ex, "SliceThickness", 1.0) or 1.0)
    meta = {
        "patient_id": getattr(ex, "PatientID", "Unknown"),
        "series_uid": uid,
        "spacing": (slice_thk, float(spacing[0]), float(spacing[1]))
    }
    return vol, meta


In [ ]:
patient_default = max(patients, key=lambda p: sum(1 for _ in p.rglob("*.dcm")))
VOL, META = load_volume_from_patient(patient_default)
print(f"Paciente elegido: {patient_default.name}")
print(f"Volumen shape (z,y,x): {VOL.shape}  |  Spacing (mm): {META['spacing']}  | Series: {META['series_uid']}")


In [ ]:
# controles de ventana
wl_slider = FloatSlider(description="WL", value=-600.0, min=-1200, max=200, step=10, continuous_update=False)
ww_slider = FloatSlider(description="WW", value=1500.0, min=200, max=3000, step=50, continuous_update=False)

def show_slice(k:int, plane="axial"):
    wl, ww = wl_slider.value, ww_slider.value
    vmin, vmax = wl - ww/2, wl + ww/2
    if plane == "axial":
        img = VOL[k]; title = f"Axial z={k}"
    elif plane == "coronal":
        img = VOL[:, k, :]; title = f"Coronal y={k}"
    else:
        img = VOL[:, :, k]; title = f"Sagital x={k}"
    plt.figure(figsize=(6,6))
    plt.imshow(img, cmap="gray", vmin=vmin, vmax=vmax)
    plt.title(f"{META['patient_id']} · {title} · WL {wl:.0f} WW {ww:.0f}")
    plt.axis("off")
    plt.show()

print("Axial")
_ = interact(lambda k: show_slice(k, "axial"),
             k=IntSlider(min=0, max=VOL.shape[0]-1, step=1, value=VOL.shape[0]//2))
display(HBox([wl_slider, ww_slider]))

print("Coronal")
_ = interact(lambda k: show_slice(k, "coronal"),
             k=IntSlider(min=0, max=VOL.shape[1]-1, step=1, value=VOL.shape[1]//2))

print("Sagital")
_ = interact(lambda k: show_slice(k, "sagittal"),
             k=IntSlider(min=0, max=VOL.shape[2]-1, step=1, value=VOL.shape[2]//2))


In [ ]:
import matplotlib.animation as animation

wl, ww = wl_slider.value, ww_slider.value
vmin, vmax = wl - ww/2, wl + ww/2

fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(VOL[0], cmap="gray", vmin=vmin, vmax=vmax)
ax.axis("off")

def update(i):
    im.set_data(VOL[i])
    return [im]

ani = animation.FuncAnimation(fig, update, frames=VOL.shape[0], interval=40, blit=True)
plt.show()
